In [3]:
import numpy as np
import pdb
from PIL import Image
from scipy import ndimage
import os
import unittest
from scipy.ndimage import maximum_filter
from numpy.lib.stride_tricks import as_strided

In [4]:
png_file = Image.open("C:\\Data\\2.png")
array_image = np.array(png_file)
kernel_3d = np.random.rand(3, 3, 4)
kernel_32d = np.random.rand(3, 3, 32)
png_file_1 = Image.open("C:\\Data\\1.png")
array_image_1 = np.array(png_file_1)
png_file_2 = Image.open("C:\\Data\\7.png")
array_image_2 = np.array(png_file_2)

In [5]:
array_image.shape

(590, 586, 4)

In [6]:
np.random.rand(2, 2)

array([[0.2323778 , 0.52880543],
       [0.48303715, 0.71540161]])

In [7]:
array_image_1.shape

(343, 345, 4)

In [8]:
test = np.random.rand(294, 292, 32)

In [9]:
to_stack = []

In [10]:
t = np.array(create_kernels(64, 32))

NameError: name 'create_kernels' is not defined

In [11]:
t.shape

NameError: name 't' is not defined

In [12]:
def reshape_dimensions_convolution(input_height, input_width, filter_height, filter_width, stride):
    output_height = int(((input_height - filter_height) / stride) + 1)
    output_width  = int(((input_width - filter_width)  / stride) + 1)
    return (output_height, output_width)

In [13]:
reshape_dimensions_convolution(3, 3, 2, 2, 1)

(2, 2)

In [14]:
def reshape_dimensions_pooling(input_height, input_width, f, s):
    output_height = int(((input_height - f) / s) + 1)
    output_width = int(((input_width - f) / s) + 1)
    return (output_height, output_width)

In [124]:
def forward_pass(image):
    shape = image.shape
    kernels = create_kernels(32, 4)
    kernels2 =  create_kernels(64, 32)
    to_stack_values = []
    dimensions = reshape_dimensions_convolution(shape[0], shape[1], 3, 3, 1) 
    for i in range(len(kernels)):
        to_stack_values.append(convolve_one_step_vectorized(image, kernels[i], 36))
    stacked_filters = np.stack(to_stack_values, axis = 2)
    first_ReLU = ReLUv2(stacked_filters)
    pool_to_reshape = reshape_dimensions_pooling(first_ReLU.shape[0], first_ReLU.shape[1], 2, 2)
    first_max_pool = np.array(max_pooling(first_ReLU)).reshape(pool_to_reshape[0], pool_to_reshape[1], 32)
    to_stack_values.clear()
    shape2 = first_max_pool.shape
    for f in range(len(kernels2)):
        to_stack_values.append(convolve_one_step_vectorized(first_max_pool, kernels2[f], 288))
    stacked_filters2 = np.stack(to_stack_values, axis = 2)
    second_ReLU = ReLUv2(stacked_filters2)
    pool_to_reshape2 = reshape_dimensions_pooling(second_ReLU.shape[0], second_ReLU.shape[1], 2, 2)
    second_max_pool = np.array(max_pooling(second_ReLU)).reshape(pool_to_reshape2[0], pool_to_reshape2[1], 64)
    fully_connected = fully_connected(second_max_pool, 120)
    classification = fully_connected(fully_connected, 10)
    softmax = More_Robust_Softmax(classification)
    return softmax
    """
    stacked_filters2 = np.stack(to_stack_values, axis = 2)
    second_ReLU = np.array(ReLUv2(stacked_filters2))
    pool_to_reshape2 = reshape_dimensions_pooling(second_ReLU.shape[0], second_ReLU.shape[1], 2, 2)
    second_max_pool = np.array(max_pooling(second_ReLU).reshape(pool_to_reshape[2], pool_to_reshape[2]), 64)
    return second_max_pool
    """

In [77]:
forward_pass(array_image)

(146, 145, 64)

In [19]:
shajp = test.shape

In [20]:
to_reshape = reshape_dimensions_convolution(shajp[0], shajp[1], 3, 3, 1)

In [21]:
alfa.shape

NameError: name 'alfa' is not defined

In [22]:
def create_kernels(number_to_create, depth):
    every_kernel = []
    for f_u in range(number_to_create):
        every_kernel.append(np.random.rand(3, 3, depth))
    return np.array(every_kernel)

In [23]:
test_image = np.random.rand(10, 10, 4)

In [24]:
def convolve_one_step(input_image, kernel_used):
    intermediate_values = []
    dimensions = input_image.shape
    dimensions_kernel = kernel_used.shape
    final_values = []
    dimensions_to_reshape = reshape_dimensions_convolution(dimensions[0], dimensions[1], dimensions_kernel[0], dimensions_kernel[1], 1)
    for i in range(dimensions[0] - 2):
        for j in range(dimensions[1] - 2):
            for k in range(dimensions[2] ):
                slice_of_image = input_image[i : i + 3, j : j + 3, k]
                real_kernel = kernel_used[:, :, k]
                value = np.vdot(slice_of_image, real_kernel)
                intermediate_values.append(value)
                #pdb.set_trace()
                if len(intermediate_values) == dimensions[2]:
                    final_values.append(sum_dimension(intermediate_values))
                    intermediate_values.clear()
    return np.array(final_values).reshape(dimensions_to_reshape)

In [25]:
def sum_dimension(list_of_three):
    return sum(list_of_three)

In [26]:
image_after_conv_2 = np.array(convolve_one_step(array_image, kernel_3d)).reshape(dimensions[0], dimensions[1], 1)

NameError: name 'dimensions' is not defined

In [42]:
def max_pooling(image_array):
    pooling_values = []
    for m in range(0, image_array.shape[0], 2):
        for n in range(0, image_array.shape[1], 2):
            for o in range(image_array.shape[2]):
                if n + 2 > image_array.shape[1]:
                    slice_of_pooling = image_array[m : m + 2, n - 1: n + 1 , o]
                else:
                    slice_of_pooling = image_array[m : m + 2, n : n + 2 , o]
                pooling_values.append(slice_of_pooling.max())
                #pdb.set_trace()
    return pooling_values

In [ ]:
filtr = test_image[0 : 2, 0 : 2, 0]

In [ ]:
after_maxpool = np.array(max_pooling(test_image)).reshape(5, 5, 4)

In [ ]:
after_maxpool.shape

In [ ]:
def ReLUv1(x):
    after_relu = []
    for r in x:
        if r > 0:
            after_relu.append(r)
        else:
            after_relu.append(0)
    return after_relu

In [39]:
def ReLUv2(x):
    return np.maximum(0, x)

In [90]:
misa = np.random.rand(145, 146, 64)

In [91]:
last_pool = np.random.rand(2, 2, 1)

In [126]:
bias = np.random.rand(120)

In [128]:
bias[0]

0.7248374207878752

In [136]:
def fully_connected(after_last_max_pool, number_of_nodes, bias):
    almost_values = []
    nodes_values = []
    shape_of_flatten = np.prod(after_last_max_pool.shape)
    flatten_array = after_last_max_pool.reshape(shape_of_flatten)
    pdb.set_trace()
    weights = np.random.rand(len(flatten_array), number_of_nodes)
    pdb.set_trace()
    for x in range(weights.shape[1]):
        for y in range(weights.shape[0]):
            if y == weights.shape[0] - 1:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
                nodes_values.append(sum(almost_values) + bias[x])
                almost_values.clear()
            else:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y] + bias[y])
    return np.squeeze(np.array(nodes_values))

In [100]:
a = np.array([])

In [125]:
np.prod((146, 145, 64))

1354880

In [101]:
fully_connected(misa, 120)

(120, 1)

In [ ]:
fully_connected(last_pool, 4)

In [ ]:
test_image = np.random.rand(10, 10, 4)

In [ ]:
test_kernel = np.random.rand(3, 3, 4)

In [43]:
def reshape_to_matrix_image(height, width, kernel_height, kernel_width, depth):
    number_of_patches = np.prod(reshape_dimensions_convolution(height, width, kernel_height, kernel_width, 1))
    each_patch_size = kernel_height * kernel_width * depth
    return (number_of_patches, each_patch_size)

In [44]:
reshape_to_matrix_image(7, 7, 3, 3, 4)

(25, 36)

In [47]:
def convolve_one_step_vectorized(imput_image, kernel_used, matmuldim):
    shape_of_image = imput_image.shape
    kernel_used_shape = kernel_used.shape
    dim_to_reshape = reshape_dimensions_convolution(shape_of_image[0], shape_of_image[1], kernel_used_shape[0], kernel_used_shape[1], 1)
    reshaped_image  = reshape_to_matrix_valid(imput_image, (3, 3), 1, -1, matmuldim)
    reshaped_kernel = np.squeeze(reshape_to_matrix_valid(kernel_used, (3, 3), 1, -1, 1))
    #pdb.set_trace()
    vectorized_output = np.matmul(reshaped_image, reshaped_kernel)
    return vectorized_output.reshape(dim_to_reshape)

In [48]:
def reshape_to_matrix_valid(image, patch_size, stride, rA, rB):
    shape = (image.shape[0] - patch_size[0] + 1,
             image.shape[1] - patch_size[1] + 1,
             patch_size[0],
             patch_size[1],
             image.shape[2])
    #pdb.set_trace()
    strides = (image.strides[0],
               image.strides[1],
               image.strides[0],
               image.strides[1],
               image.strides[2])
    patches = as_strided(image, shape, strides)
    #reshaped_patches = patches.reshape(rA, rB)
    return patches.reshape(rA, rB)

In [29]:
reshape_to_matrix_valid(array_image, (3, 3), 1, 9, -1)

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ...,  32,  32, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 174, 174, 255],
       [235, 235, 235, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [30]:
reshape_dimensions_convolution(5, 5, 3, 3, 1)

(3, 3)

In [31]:
a = convolve_one_step_vectorized(array_image, test_kernel)

NameError: name 'test_kernel' is not defined

In [32]:
b = convolve_one_step(array_image, test_kernel)

NameError: name 'test_kernel' is not defined

In [33]:
a == b

NameError: name 'a' is not defined

In [34]:
test = np.random.rand(294, 292, 32)

In [35]:
reshape_to_matrix_valid(test, (3, 3), 1, -1, 36)

array([[0.20681253, 0.80835766, 0.38597403, ..., 0.01517015, 0.31011848,
        0.71930138],
       [0.65423651, 0.43844318, 0.33000965, ..., 0.89121916, 0.81869301,
        0.51871138],
       [0.57388651, 0.26497996, 0.94762847, ..., 0.97711944, 0.59958313,
        0.89740574],
       ...,
       [0.37852654, 0.85882519, 0.22913201, ..., 0.13118685, 0.10113197,
        0.7171832 ],
       [0.41338112, 0.6861293 , 0.743424  , ..., 0.4350309 , 0.13967408,
        0.71134418],
       [0.62819449, 0.64891033, 0.83085349, ..., 0.52659828, 0.34193572,
        0.70928081]])

In [113]:
def Softmax(values):
    numerator = np.exp(values)
    denominator = np.sum(numerator)
    return numerator / denominator

In [121]:
def More_Robust_Softmax(values):
    numerator = np.exp(values - np.max(values))
    denominator = np.sum(numerator)
    return numerator / denominator

In [103]:
ol = np.array([5, 4, -1])

In [104]:
np.exp(ol)

array([148.4131591 ,  54.59815003,   0.36787944])

In [130]:
a = np.random.rand(120)

In [107]:
a.shape

(120,)

In [ ]:
b = fully_connected(a, 10, bias)

> c:\users\dembek\appdata\local\temp\ipykernel_18428\606380187.py(7)fully_connected()

ipdb> p flatten_array[0]
0.8363508852274629
ipdb> p flatten_array.shape
(120,)
ipdb> c
> c:\users\dembek\appdata\local\temp\ipykernel_18428\606380187.py(9)fully_connected()



In [119]:
Softmax(b)

array([0.00045416, 0.27799541, 0.10229085, 0.17043475, 0.02124742,
       0.02601075, 0.23646179, 0.00251889, 0.01909501, 0.14349097])

In [133]:
b

array([89.31919571, 92.25376027, 91.77294417, 89.41211037, 92.92622867,
       89.8564207 , 91.74248784, 91.07681137, 90.93248558, 90.95229806])